In [1]:
!pip3 install datasets evaluate transformers
!pip3 install faiss-cpu
!pip install -U sentence-transformers
!pip3 install openai

In [2]:
from transformers import AutoTokenizer, AutoModel
from transformers.pipelines import pipeline
from datasets import load_dataset, Dataset

In [3]:
from transformers.pipelines import pipeline

question_answerer = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
from datasets import Dataset
resume_dataset = load_dataset("csv", data_files='ResumeDataSet.csv', split="train")
resume_dataset
print (resume_dataset)

Using custom data configuration default-b99ae1db88696035
Found cached dataset csv (/Users/luissanjuan/.cache/huggingface/datasets/csv/default-b99ae1db88696035/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Category', 'Resume'],
    num_rows: 962
})


In [5]:
resume_dataset.set_format("pandas")
df = resume_dataset[:]

In [6]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
from datasets import Dataset

resume_dataset = Dataset.from_pandas(df)
resume_dataset

Dataset({
    features: ['Category', 'Resume'],
    num_rows: 962
})

In [8]:
resume_dataset = resume_dataset.map(
    lambda x: {"resume_length": len(x["Resume"].split())}
)

  0%|          | 0/962 [00:00<?, ?ex/s]

In [9]:
resume_dataset = resume_dataset.filter(lambda x: x["resume_length"] > 15)
resume_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['Category', 'Resume', 'resume_length'],
    num_rows: 962
})

In [10]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [11]:
import torch

device = torch.device('cpu')
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0): MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_features

In [12]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [13]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [14]:
embedding = get_embeddings(resume_dataset["Resume"][0])
embedding.shape

torch.Size([1, 768])

In [15]:
embeddings_dataset = resume_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["Resume"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/962 [00:00<?, ?ex/s]

In [16]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Category', 'Resume', 'resume_length', 'embeddings'],
    num_rows: 962
})

In [17]:
question = '''
Startup is looking for a founder engineer with experience on data analist, Experience in frontEnd development with Angular is also desired...
'''
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [18]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

In [19]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [20]:
for _, row in samples_df.iterrows():
    print(f"CATEGORY: {row.Category}")
    print(f"SCORE: {row.scores}")
    print(f"RESUME: {row.Resume}")
    print("=" * 10)
    print()

CATEGORY: Data Science
SCORE: 32.55263900756836
RESUME: Personal Skills â¢ Ability to quickly grasp technical aspects and willingness to learn â¢ High energy levels & Result oriented. Education Details 
January 2018 Master of Engineering Computer Technology & Application Bhopal, Madhya Pradesh Truba Institute of Engineering & Information Technology
January 2010 B.E. computer science Bhopal, Madhya Pradesh RKDF Institute of Science and Technology College of Engineering
January 2006 Polytechnic Information Technology Vidisha, Madhya Pradesh SATI Engineering College in Vidisha
January 2003 M.tech Thesis Detail  BMCH School in Ganj basoda
Data science 

I have six month experience in Data Science. Key Skills: - Experience in Machine Learning, Deep Leaning, NLP, Python, SQL, Web Scraping Good knowledge in computer subjects and ability to update
Skill Details 
Experience in Machine Learning, Deep Learning, NLP, Python, SQL, Web Crawling, HTML,CSS.- Exprience - Less than 1 year monthsCompan